# Detección de salto de línea de lectura en tiempo real

## Importaciones

In [ ]:
from keras.models import load_model
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Definiciones y métodos

In [ ]:
def detecta_ojo(frame, face_cascade, eye_cascade, ojo_a_analizar="D", restringir_detectar_dos_ojos=True):
    """Devuelve la imagen del ojo derecho encontrado, en otro caso, devuelve el frame."""
    
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)

    if len(faces) > 0:
        x, y, w, h = faces[0]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 5)
        face_img = frame[y:y+w, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_img, 1.3, 5)

        if len(eyes) < 2 and restringir_detectar_dos_ojos: # Si no detecta los 2 ojos, que coja el frame anterior.
            return (False, frame)

        if len(eyes) > 0:
            
            pos_ojo = get_posicion_ojo(ojo_a_analizar, eyes, faces[0])

            ex, ey, ew, eh = eyes[pos_ojo]
            eye_img = face_img[ey:ey+eh,ex:ex+ew]

            return (True, eye_img)

    return (False, frame)

In [ ]:
def get_posicion_ojo(tipo_ojo, ojos_detectados, cara):
    """Devuelve la posición del ojo deseado (izquierdo en el vídeo o derecho) en ojos_detectados."""
    
    x, y, w, h = cara
    pos_ojo=0
    x_ojo=0

    isOjoALaDerecha = tipo_ojo == "D"
    isOjoALaIzquierda = tipo_ojo == "I"

    if isOjoALaIzquierda:
        x_ojo=x+w

    for i in range(len(ojos_detectados)):

        if isOjoALaIzquierda and ojos_detectados[i][0] < x_ojo:
            x_ojo=ojos_detectados[i][0]
            pos_ojo=i

        if isOjoALaDerecha and ojos_detectados[i][0] > x_ojo:
            x_ojo=ojos_detectados[i][0]
            pos_ojo=i

    return pos_ojo

In [ ]:
def hasBreakLine(modelo_final, frames_ojo_list):
    '''Devuelve true si se trata de una línea leída.'''
    
    x_test_prueba_set = np.array([frames_ojo_list])
    
    results = np.argmax(modelo_final.predict(x_test_prueba_set), axis = 1)

    return results[0] == 1

In [ ]:
def getImageShape(image):
  """Devuelve las dimensiones de la imagen."""
  
  height, width = image.shape[:2]

  return height, width

In [ ]:
def changeImageColor(target_img):
  """Devuelve una imagen con un filtro de color aplicado."""
  
  #target_img = cv2.cvtColor(target_img,cv2.COLOR_GRAY2RGB) # Aplicar solo si la imagen de entrada está en escala de grises.
  
  # Creamos dos copias de la imagen:
  # · Una para la capa de encima.
  # · Otra para la salida.
  overlay = target_img.copy()
  output = target_img.copy()

  img_h, img_w = getImageShape(target_img)

  # Dibujamos un rectángulo en la imagen.

  cv2.rectangle(overlay, (0, 0), (img_w, img_h), (0, 255, 0), -1)
  
  # Aplicamos la capa.
  cv2.addWeighted(overlay, 0.4, output, 1 - 0.4, 0, output)

  return output


In [ ]:
def addInfoLineasLeidas(target_img, numLineasLeidas, segundoEnCurso):
    """Devuelve una imagen con la info de líneas leidas y segundos del vídeo."""

    # Creamos dos copias de la imagen:
    # · Una para la capa de encima.
    # · Otra para la salida.
    overlay = target_img.copy()
    output = target_img.copy()

    blue=188.955
    green=113.985
    red=0

    font_size=0.8
    font_weight=2

    # Añadimos el texto.
    cv2.putText(overlay, f"{numLineasLeidas} LINEAS LEIDAS EN {segundoEnCurso} SEGUNDOS", 
                (20, 60), cv2.FONT_HERSHEY_COMPLEX, font_size, (blue, green, red), font_weight)

    cv2.putText(overlay, f"TASA: {tasaLeidoSegundo(numLineasLeidas, segundoEnCurso)} LINEAS/SEGUNDO", 
                (20, 100), cv2.FONT_HERSHEY_COMPLEX, font_size, (blue, green, red), font_weight)
    
    alpha=0.8

    # Aplicamos la capa.
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    return output

In [ ]:
def cargaVideo(video_path):
  '''Obtiene el vídeo, los FPS y la duración.'''

  video = cv2.VideoCapture(video_path)
  fps = video.get(cv2.CAP_PROP_FPS)
  frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
  seconds = int(frames / fps)

  width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

  return video, fps, seconds, width, height

In [ ]:
def reescalaFrame(frame):
    """Devuelve el frame reescalado para el trabajo con la red."""

    frame_reescalado = cv2.resize(frame,(800,600),fx=0,fy=0, interpolation = cv2.INTER_AREA)

    return frame_reescalado

In [ ]:
def reescalaOjo(frame_ojo):
    '''Devuelve el frame del ojo reescalado para el trabajo con el modelo.'''

    frame_ojo_reescalado = cv2.resize(frame_ojo, (80, 80), interpolation=cv2.INTER_AREA)  

    return frame_ojo_reescalado  

In [ ]:
def frameToGrayScale(frame):
    """Devuelve el frame en escala de grises para el trabajo con la red."""

    frame_escala_grises = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    return frame_escala_grises

In [ ]:
def framesADescartar(fps, num_frames_borrar):
    """Devuelve un listado de frames a descartar del video."""
    
    framesDescartados = set()

    if num_frames_borrar!=0:
        periodo = round(fps) / num_frames_borrar

        for i in range(num_frames_borrar):
            frame_descartado=round(i*periodo)
            framesDescartados.add(frame_descartado)

    return framesDescartados

In [ ]:
def tasaLeidoSegundo(numLineasLeidas, segundoEnCurso):
    """Devuelve la tasa de líneas leídas por segundo."""

    tasa = segundoEnCurso
    
    if segundoEnCurso != 0:
        tasa = round(numLineasLeidas/segundoEnCurso, 3)

    return tasa

In [ ]:
def generateReadingRateChart(numLineasLeidas, lineas_por_segundo, totalSegundos):
    """Devuelve el gráfico de líneas/segundo."""
    
    # Creamos la figura que contendrá el gráfico.
    fig = plt.figure()

    # Gráfico de evolucion del rendimiento del lector.
    x = range(len(lineas_por_segundo))
    plt.plot(x, lineas_por_segundo, label='Líneas leídas/segundo')
    plt.legend(loc="lower right")

    plt.ylim(0, 1)
    
    # Hacemos que muestre todos los segundos del vídeo en el gráfico, 
    # aunque no se le estén pasando todas las x todavía.
    plt.xlim([0, totalSegundos])
    plt.xticks((range(totalSegundos)[0::10]))

    # Convertimos el gráfico al tipo de imagen admitida por OpenCV: numpy array.
    canvas = FigureCanvas(fig)
    canvas.draw()

    # Convertimos el cancas a una imagen.
    graph_image = np.array(fig.canvas.get_renderer()._renderer)

    # La imagen es del tipo RGB. OpenCV trabaja por defecto con BGR, por lo que la convertimos.
    chart_image = cv2.cvtColor(graph_image,cv2.COLOR_RGB2BGR)
    
    # Cerramos la figura para prevenir llenar la memoria.
    plt.close(fig)

    return chart_image

In [ ]:
def addImageToFrame(current_frame, image_watemark):
    """Añade una imagen superpuesta al frame de entrada."""

    # Escalamos la imagen de entrada.
    watemark_scale = 70

    wm_width = int(image_watemark.shape[1] * watemark_scale/100)
    wm_height = int(image_watemark.shape[0] * watemark_scale/100)
    wm_dim = (wm_width, wm_height)

    resized_wm = cv2.resize(image_watemark, wm_dim, interpolation=cv2.INTER_AREA)

    img_height, img_width = getImageShape(resized_wm)

    # Establecemos las coordenadas donde meter la imagen superpuesta.
    top_y = 500
    left_x = 20
    bottom_y = top_y + img_height
    right_x = left_x + img_width

    # Añadimos la imagen al frame.
    current_frame[top_y:bottom_y, left_x:right_x] = resized_wm

    return current_frame

## Ejecución del programa

In [ ]:
# datetime object containing current date and time
now = datetime.now()

# Parámetros de entrada.
video_path = "/path/Saturdays AI - Equipo ojo/Videos/v_persona_01.mp4"
ai_model_path = '/path/Saturdays AI - Equipo ojo/modeloEntrenado'
video_output_path = f'/path/Saturdays AI - Equipo ojo/video_results/video_output-{now.strftime("%d-%m-%Y_%H-%M-%S")}.mp4'

In [ ]:
# Cargamos el modelo con el que predecir los saltos de línea.
modelo_final = load_model(ai_model_path)

In [ ]:
# Cargamos los clasificadores.
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Obtenemos el vídeo y metadatos.
video, fps, segundos, width, height = cargaVideo(video_path)

# Reescalamos el vídeo a los fps de la red.
num_frames_borrar=round(fps)-10
framesDescartados = framesADescartar(fps, num_frames_borrar)

# Establecemos el vídeo de salida con los mismos parámetros que el de entrada.
video_output = cv2.VideoWriter(video_output_path, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 10.0, (width, height))

# Variables de entorno.
frame_ojo_prev=[]
frames_ojo_list=[]
frames_list=[]
frame_counter = 0
num_frame_por_segundo=0
num_segundo=0
num_lineas_leidas=0
lines_per_second=[]

while video.isOpened():

    hasFrame, frame = video.read()

    if not hasFrame:
      break

    print(f"\rVoy por el frame {frame_counter} de {(segundos*fps)}.", end="")
    
    frame_counter+=1

    num_frame_por_segundo=(num_frame_por_segundo+1)%round(fps)

    if num_frame_por_segundo in framesDescartados:
      continue
    
    frame_reescalado = reescalaFrame(frame)
    frame_escala_grises = frameToGrayScale(frame_reescalado)

    ojo_detectado, frame_ojo = detecta_ojo(frame_escala_grises, face_cascade, eye_cascade)

    # Si no obtenemos ojo en este frame, recuperamos el anterior.
    if not ojo_detectado and len(frame_ojo_prev) > 0: 
        frame_ojo = frame_ojo_prev

    frame_ojo_reescalado = reescalaOjo(frame_ojo)

    frames_ojo_list.append(frame_ojo_reescalado)
    frames_list.append(frame)

    # Acumulamos los 10 frames con los que trabaja el modelo.
    if len(frames_ojo_list) == 10:

        isBreakLine = hasBreakLine(modelo_final, frames_ojo_list)
        
        if isBreakLine:
            num_lineas_leidas+=1

        for i in range(len(frames_ojo_list)):
          current_frame = frames_list[i]

          # Realizamos las modificaciones sobre el frame.
          if isBreakLine:
            current_frame = changeImageColor(current_frame)
                    
          current_frame = addInfoLineasLeidas(current_frame, num_lineas_leidas, num_segundo)

          lines_per_second_chart = generateReadingRateChart(num_lineas_leidas, lines_per_second, segundos)

          current_frame = addImageToFrame(current_frame, lines_per_second_chart)

          # Grabamos el frame en el nuevo vídeo.
          video_output.write(current_frame)

        # Reseteamos variables de entorno.
        frames_ojo_list = []
        frames_list = []
        lines_per_second.append(tasaLeidoSegundo(num_lineas_leidas, num_segundo))
        num_segundo+=1

    frame_ojo_prev = frame_ojo

# Finalizamos ejecución y guardamos el vídeo
video.release()
video_output.release()
cv2.destroyAllWindows()